# 画像データとメタデータから、Pawpularityを予測する。

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import os
import matplotlib.pyplot as plt

from tqdm import tqdm
import gc

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
DIR_PATH = "/kaggle/input/petfinder-pawpularity-score/"
IMG_SIZE = 128

In [ ]:
def get_train_file_path(id):
    return f"{DIR_PATH}train/{id}.jpg"

df = pd.read_csv(f"{DIR_PATH}train.csv")
df['file_path'] = df['Id'].apply(get_train_file_path)

In [ ]:
"""
!pip install timm
import sys
sys.path.append("../input/module")
from transfer_learning import make_image_feature
num_classes = 1000

# global average は全結合に行くとこしかない。
features = make_image_feature(df['Id'], num_classes=num_classes)

col = ["Id"]
for i in range(num_classes):
    col.append(f"img_feature{i+1}")
df_img_feature = pd.DataFrame(features, columns=col)
df_img_feature.to_csv(f"/kaggle/working/pre_output.csv")
df_all = pd.merge(df, df_img_feature, on="Id")
df_all.to_csv(f"/kaggle/working/train_merged.csv")

del features
gc.collect()
"""
pass # セルの出力がうっとうしい

In [ ]:
#df_all = pd.read_csv("/kaggle/input/petfinder-data/train_merged.csv", index_col=0)
#df_all.head()

df_img_feature = pd.read_csv("/kaggle/input/petfinderdata/pre_output.csv", index_col=0)
df_all = pd.merge(df, df_img_feature, on="Id")
df_all.head()

In [ ]:
# normalization
def normalization(series):
    if series.name not in ['Id', 'Pawpularity', 'file_path']:
        series = (series - series.mean()) / series.std()
    return series

df_all = df_all.apply(normalization)

In [ ]:
class PetDataSet(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        feature_cols = [col for col in df.columns if col not in ['Id', 'Pawpularity', 'file_path']]
        self.targets = df['Pawpularity'].values
        self.data = df[feature_cols].values
        self.transforms = transforms # <-読み出し時に前処理をするやつ
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        meta = self.data[index, :12]
        im_feature = self.data[index, 12:]
        target = self.targets[index]
        
        if self.transforms:
            pass
            
        return im_feature.astype('float32'), meta.astype('float32'), target.astype('float32')

In [ ]:
# 10出力のEfficientNetで画像から特徴量を出して、metadataとcatして推論
# cat以降のネットワーク(線形モデル)だけ学習
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        #self.flatten = nn.Flatten()
        self.image_layer = nn.Sequential(
            #nn.Dropout(0.2),
            nn.Linear(1000, 32),
            nn.ReLU(),
            nn.Linear(32, 20)
        )
        self.output_layer = nn.Sequential(
            #nn.Dropout(0.2),
            nn.Linear(20+12, 8),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x_im, x_meta):
        #x = self.flatten(x)
        im_out = self.image_layer(x_im)
        connected = torch.cat([im_out, x_meta], axis=1)
        output = self.output_layer(connected) * 100
        return output
    
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

## construction

In [ ]:
# coustruct model
ai = NeuralNet().to(device)
ai.apply(init_weights)

# make train, validate dataset
ds = PetDataSet(df_all)
n_samples = len(ds) 
train_size = int(n_samples * 0.9) 
val_size = n_samples - train_size

train_dataset, val_dataset = torch.utils.data.random_split(ds, [train_size, val_size])

In [ ]:
BATCH_SIZE = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = len(val_dataset), shuffle = False, num_workers = 2)
full_train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = len(train_dataset), shuffle = False, num_workers = 2)

## training

In [ ]:
# mati modalだとうまくいきにくい
def train_one_epoch(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    mean = 0
    for n, (x_im, x_meta, target) in enumerate(dataloader):
        x_im = x_im.to(device)
        x_meta = x_meta.to(device)
        target = target.unsqueeze(1).to(device)
        pred = model(x_im, x_meta)
        loss = loss_fn(pred, target)
        mean += loss / BATCH_SIZE
        
        optimizer.zero_grad() # 前回計算した勾配をクリア
        loss.backward()
        optimizer.step()
        
    return mean

def validation(val_loader, model, mess=True):
    for x_im, x_meta, target in val_loader:
        x_im = x_im.to(device)
        x_meta = x_meta.to(device)
        target = target.unsqueeze(1).to(device)
        pred = model(x_im, x_meta)
        loss = loss_fn(pred, target)

        return loss.item()

In [ ]:
epochs =  100
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(ai.parameters(), lr=1e-3, weight_decay=30)
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

log = []
val_log = []
valid = True
ai.train()
for i in tqdm(range(epochs)):
    loss = train_one_epoch(train_loader, ai, loss_fn, optimizer)
    #scheduler.step()
    log.append(loss)
    if valid:
        val_log.append(validation(val_loader, ai, mess=False))
    # print(f"{i}_th epoch done: loss = {loss}")

In [ ]:
plt.title("MSE")
plt.xlabel("epochs")
plt.plot(log, label="train_loss")
plt.plot(val_log, label="val_loss")
plt.legend()
plt.show()

## validation

In [ ]:
ai.eval()
print("MSE =", validation(val_loader, ai)) # 22点くらい間違ってる 484

## consideration

In [ ]:
ai.eval()
for x_im, x_meta, target in val_loader:
    x_im = x_im.to(device)
    x_meta = x_meta.to(device)
    target = target.unsqueeze(1).to(device)
    pred = ai(x_im, x_meta)
    
n_pred = pred.to("cpu").detach().numpy().copy()
n_target = target.to("cpu").detach().numpy().copy()

In [ ]:
# y=xになってたらいい

plt.figure(figsize=(5,5))
plt.scatter(n_target, n_pred, marker=".")
plt.xlabel("true")
plt.ylabel("pred")
plt.xlim(0,100)
plt.ylim(0,100)
plt.show()

In [ ]:
er = abs(n_pred-n_target)
print(np.mean(er))
plt.hist(er, bins=100)
plt.xlim(0,100)
plt.show()

# test

In [ ]:
"""
!pip install timm
import timm

def test_image_feature(id_series, num_classes=10, IMG_SIZE=128):
    pretrained = timm.create_model('efficientnet_b5', pretrained=True, num_classes=num_classes)
    pretrained = pretrained.to(device)
    features = []
    for i, path in tqdm(enumerate(os.listdir("/kaggle/input/petfinder-pawpularity-score/test"))):
        Id = path.split(".")[0]
        path = "/kaggle/input/petfinder-pawpularity-score/test/" + path
        img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype('float32') / 255.
        img = img.reshape((1,)+img.shape)
        img = torch.from_numpy(img).clone()
        img = img.permute(0,3,1,2)
        img = img.to(device)
        feature = pretrained(img)[0].tolist()
        feature.insert(0, Id)
        features.append(feature)
        del img, feature
        gc.collect()

    return features

num_classes = 1000

# global average は全結合に行くとこしかない。
features = test_image_feature(df['Id'], num_classes=num_classes)
col = ["Id"]
for i in range(num_classes):
    col.append(f"img_feature{i+1}")
test_img_feature = pd.DataFrame(features, columns=col)
test_img_feature.to_csv("/kaggle/working/test_img_feature.csv", index=False)
"""
pass

In [ ]:
test_img_feature = pd.read_csv("/kaggle/input/test-data/test_img_feature.csv")
test_img_feature.head()

In [ ]:
test_df = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/test.csv")
test_df["Pawpularity"] = np.nan
test_df = pd.merge(test_df, test_img_feature, on="Id")
test_df.head()

In [ ]:
test_dataset = PetDataSet(test_df)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = len(test_dataset), shuffle = False, num_workers = 2)

In [ ]:
def test(test_loader, model):
    for x_im, x_meta, target in test_loader:
        x_im = x_im.to(device)
        x_meta = x_meta.to(device)
        pred = model(x_im, x_meta)
        
    return pred

In [ ]:
evaluation = test(test_loader, ai).to("cpu").detach().numpy().copy()
evaluation = pd.DataFrame(np.vstack((test_df["Id"].values, evaluation[:,0])).T, columns=["Id", "Pawpularity"])
evaluation.Pawpularity = evaluation.Pawpularity.astype(float)
evaluation = evaluation.round({"Pawpularity":2})

In [ ]:
evaluation.to_csv("/kaggle/working/submission.csv", index=False)